In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
from settings import *
from math import sqrt

# import xgboost as xgb
from sklearn import preprocessing, neighbors, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
  RandomForestRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

DIR_DATA = "data"
DIR_PROCESSED = "processed"
LABEL = "rating"
NON_PREDICTORS = [LABEL, "int_rating"]#, "name","anime_id"]
CV_FOLDS = 5

In [2]:
a_file = open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle', 'rb')
anime = pickle.load(a_file)
a_file.close()

# TODO: Deal with nan genre in anime

# "rating" is in both tables
anime["avg_rating"] = anime.rating
anime = anime.drop("rating", 1)

ratings = pd.read_csv(DIR_DATA + '/rating.csv')

In [3]:
shows = pd.merge(anime, ratings)
shows.head()

,anime_id,name,episodes,members,Movie,Music,ONA,OVA,Special,TV,...,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan,int_rating,avg_rating,user_id,rating
0,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,99,5
1,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,152,10
2,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,244,10
3,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,271,10
4,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,278,-1


In [4]:
userRatings = shows.pivot_table(index=['anime_id'], columns='user_id', values='rating')
userRatings.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,10.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,10.0,NaN
6,NaN,NaN,NaN,-1.0,8.0,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
a_file = open(DIR_PROCESSED + '/sparse_merged_user_pivot_table.pickle', 'wb')
pickle.dump(userRatings.to_sparse(), a_file)
a_file.close()


In [16]:
from scipy.sparse import csr_matrix
userMatrix = userRatings.replace(np.nan, 0)
userMatrix = userMatrix.replace(-1, 0)
print(userMatrix)

MemoryError: 

In [ ]:
userRatings.head()